In [ ]:
import numpy as np
import math
import plotly.express as px
import pandas as pd
import pickle
pi = math.pi
sqrt = math.sqrt

In [ ]:
import generate_data
import algorithm
from utils import *
from plots import *

### 设置参数

In [ ]:
sigma=0.1
n_eigenvectors=50
n_factors=2
eig_crit=delta=0.5
sim_crit=gamma=0.85
K=0

### 生成数据

In [ ]:
data = generate_data.generate_synthetic_data([2 ,3 ,0.1], 10000, 'rectangle3d', noise=0.05)
data.shape
#data = generate_data.generate_cryo_em_data(1000, 20, 0, 10000, 0)

In [ ]:
df = pd.DataFrame(data, columns=['x','y','z'])
range_axes = [-1,4]
fig = px.scatter_3d(df, x='x', y='y', z='z',
                    range_x=range_axes, range_y=range_axes, range_z=range_axes,
                    width=600, height=400)
fig.update_traces(marker_size = 1)
fig

### 距离矩阵

In [ ]:
W = calc_W(data, sigma)
W.shape

### 特征向量和特征值

In [ ]:
phi, Sigma = calc_vars(data, W, sigma, n_eigenvectors)
phi.shape, Sigma.shape

In [ ]:
best_matches, best_sims, all_sims = find_combos(phi, Sigma, n_factors, eig_crit, sim_crit)

In [ ]:
print(best_matches)
print(best_sims)

### 分解

In [ ]:
labels, C = split_eigenvectors(best_matches, best_sims, n_eigenvectors, K, n_factors)
labels.shape, C.shape

In [ ]:
manifolds = []
for m in range(n_factors):
    manifold = labels[0][np.where(labels[1]==m)[0]]
    manifolds.append(manifold)
for idx,m in enumerate(manifolds):
    if 1 in m:
        m1 = manifolds.pop(idx)
        manifolds.insert(0, m1)
manifolds

### 画图

In [ ]:
df = pd.DataFrame(np.hstack([data,phi]), 
                  columns=['x','y','z']+['%d-th eigen'%i for i in range(phi.shape[1])])
range_axes = [-1,4]
i=4
print('lambda =',Sigma[i])
fig = px.scatter_3d(df, x='x', y='y', z='z', color='%d-th eigen'%i,
                    range_x=range_axes, range_y=range_axes, range_z=range_axes,
                    width=600, height=400)
fig.update_traces(marker_size = 1)
fig

In [ ]:
i = manifolds[0][0]
j = manifolds[0][1]
df = pd.DataFrame(phi, columns=['%d-th eigen'%i for i in range(phi.shape[1])])
fig = px.scatter(df, x='%d-th eigen'%i, y='%d-th eigen'%j, 
                 width=600, height=400)
fig

In [ ]:
i = manifolds[1][0]
j = manifolds[1][1]
df = pd.DataFrame(phi, columns=['%d-th eigen'%i for i in range(phi.shape[1])])
fig = px.scatter(df, x='%d-th eigen'%i, y='%d-th eigen'%j, 
                 width=600, height=400)
fig